In [6]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import random
import math
import networkx as nx
import json
import numpy as np
import re
from shapely.geometry import MultiLineString


In [9]:
class GraphConstruction:
    def __init__(self,shp, input_Folder, output_folder):
        object.__init__(self)
        
        self.input_Folder = input_Folder
        self.output_folder = output_folder
        self.G = nx.Graph()
        self.shp = shp
        self.initVariables()

    def initVariables(self):

        self.shp.nodes_path = "Data SIG/" + self.shp.dataset + "/"+self.input_Folder+"/Nodes.shp"  # Path to the nodes shapefile
        self.shp.pipes_path = "Data SIG/" + self.shp.dataset + "/"+self.input_Folder+"/Pipes.shp"  # Path to the pipes shapefile
        

        self.nodes_output_file = f"Data SIG/{self.shp.dataset}/{self.output_folder}/Nodes.json"
        self.edges_output_file = f"Data SIG/{self.shp.dataset}/{self.output_folder}/Pipes.json"

        self.infIdDummy = 100000
        self.supIdDummy = 110000

        self.DummyNodes = {}
        self.nodes_data = []
        self.edges_data = []
        
        self.mapping_color_components = {
            "red": "Manholes", "springgreen": "Structures", "yellow": "Pumps",
            "cyan": "Fittings", "black": "TreatmentPlant", "orange": "Accessories",
            "violet": "Dummy", "blue": "Appariel", "bisque": "Deversoir", "peru": "PosteRefoulement"
        }


    def construct(self, AllPipes = True, idPipe = 100, insertBuffer = False):

        self.shp.read_shapefile("Pipes")
        self.shp.read_shapefile("Nodes")
        self.shp.read_shapefile("Buffers")

        # Get the minimum non-null and non-zero diameter value
        min_diameter_m = self.shp.get_min_diameter_pipes()
        default_diameter = 200
        spatial_index = self.shp.gdfNodes.sindex
        for index, row in self.shp.gdfPipes.iterrows():
            intersection = []
            geometryPipe = row['geometry']
            if geometryPipe != None:
                if row['id'] == idPipe or AllPipes:
                    lines = []
                    if geometryPipe.geom_type == 'LineString':
                        lines.append(geometryPipe)
                        first_point = geometryPipe.coords[0]
                        last_point = geometryPipe.coords[-1]

                    first_point_object = Point(first_point)
                    last_point_object = Point(last_point)

                    diameter = self.shp.get_diameter_pipe(row['id'])
                    if diameter is None or math.isnan(diameter) or diameter == 0:
                        bufferDistance = default_diameter/2
                    else:
                        bufferDistance = diameter/2

                    bufferDistance = bufferDistance / 100.0

                    buffer_geometry = geometryPipe.buffer(bufferDistance)
                    
                    if insertBuffer:
                        self.shp.insert_buffer(buffer_geometry)

                    # Use spatial index to speed up the within query
                    possible_matches_index = list(spatial_index.intersection(buffer_geometry.bounds))
                    possible_matches = self.shp.gdfNodes.iloc[possible_matches_index]

                        # Filter the possible matches using the actual within check
                    elements_inside_polygon = possible_matches[possible_matches.geometry.within(buffer_geometry)]

                    for indexNode, rowNode in elements_inside_polygon.iterrows():
                        dic = {}
                        geometryNode = rowNode['geometry']
                        distance = first_point_object.distance(geometryNode)
                            
                        dic["id"] = rowNode['id']
                        dic["type"] = "node"
                        dic["distance"] = distance
                        dic["type_1"] = rowNode["source_1"]
                        dic["position"] = [geometryNode.x, geometryNode.y]
                        intersection.append(dic)
                    #print(intersection)
                    distanceMax = first_point_object.distance(last_point_object)

                    #print("distanceMax :", distanceMax)
                    intersection = self.addDummyNodes(intersection, first_point, last_point, distanceMax , bufferDistance)

                    #print("Dummy ", intersection)
                    sorted_intersection = {}              
                    sorted_intersection = sorted(intersection, key=lambda x: x['distance'])
                    #print("sorted_intersection ", sorted_intersection)
                    intersectionFinal = self.add_extrimities(sorted_intersection, first_point, last_point, distanceMax, bufferDistance)

                    #print("intersectionFinal ", intersectionFinal)
                    self.addInGraph(intersectionFinal, row)

                        #print(self.G)

                '''else:
                    print("Skipping id:", row['id'])'''
            else:
                print("id:", row['id'], " is a none object.")
        return True

    def addDummyNodes(self, intersection, first_point, last_point, distanceMax, bufferDistance):
        if len(intersection) == 0:
            intersection = self.AddTwoDummyNodes(intersection, first_point, last_point, distanceMax)
        elif len(intersection) == 1:
            intersection = self.AddOneDummyNode(intersection, first_point, last_point, distanceMax, bufferDistance)
        return intersection
    #thsi function when we dont have any intersection so at the minumun we must to have to dummy nodes in the extrimities
    def AddTwoDummyNodes(self, intersection, first_point, last_point, distanceMax):

        randomId = self.generate_unique_random_number()
        position = [first_point[0], first_point[1]]


        for key, value in self.DummyNodes.items():
            if value == position:
                randomId = key


        self.DummyNodes[randomId] = position
        intersection.insert(0,{'id':randomId, 'type': 'node','type_1': 'Dummy' ,'distance':0, 'position':position})



        randomId = self.generate_unique_random_number()
        position = [last_point[0], last_point[1]]
        for key, value in self.DummyNodes.items():
            if value == position:
                randomId = key
        self.DummyNodes[randomId] = position
        intersection.append({'id':randomId, 'type': 'node','type_1': 'Dummy' , 'distance':distanceMax, 'position':position})

        return intersection
    
    def AddOneDummyNode(self, intersection, first_point, last_point, distanceMax, bufferDistance):
        Test = True
        Test2 = True
        alpha = bufferDistance
    
        
        if Test:
            if last_point[0] - alpha <= intersection[-1]['position'][0] <= last_point[0] + alpha and last_point[1] - alpha <= intersection[-1]['position'][1] <= last_point[1] + alpha:

            #if liste[-1]['position'] == [last_point[0],last_point[1]]:
       
                randomId = self.generate_unique_random_number()
                position = [first_point[0], first_point[1]]
                for key, value in self.DummyNodes.items():
                    if value == position:
                        randomId = key
                self.DummyNodes[randomId] = position
                intersection.insert(0, {'id':randomId, 'type': 'node', 'type_1': 'Dummy' ,'distance':0, 'position':position})
                Test2 = False
        if Test and Test2:
            if first_point[0] - alpha <= intersection[0]['position'][0] <= first_point[0] + alpha and first_point[1] - alpha <= intersection[0]['position'][1] <= first_point[1] + alpha:
            #if liste[0]['position'] == [first_point[0],first_point[1]]:
                randomId = self.generate_unique_random_number()
                position = [last_point[0], last_point[1]]
                for key, value in self.DummyNodes.items():
                    if value == position:
                        randomId = key
                self.DummyNodes[randomId] = position
                intersection.append({'id':randomId, 'type': 'node', 'type_1': 'Dummy' ,'distance':distanceMax, 'position':position})


        return intersection
  
    def generate_unique_random_number(self):

        while True:
            random_number = random.randint(self.infIdDummy, self.supIdDummy)
            if random_number not in self.DummyNodes:
                return random_number

    def add_extrimities(self, sorted_intersection, first_point, last_point, distanceMax, bufferDistance):
        
        alpha = bufferDistance
        
        distance1 = self.euclidean_distance(first_point, sorted_intersection[0]['position'])
        distance2 = self.euclidean_distance(first_point, sorted_intersection[-1]['position'])
        
        distance3 = self.euclidean_distance(last_point, sorted_intersection[-1]['position'])
        distance4 = self.euclidean_distance(last_point, sorted_intersection[0]['position'])
        
        '''print("distance 1", distance1)
        print("distance 2", distance2)
        print("distance 3", distance3)
        print("distance 4", distance4)'''
        
        if distance1 > alpha and distance2 > alpha:

            randomId = self.generate_unique_random_number()
            position = [last_point[0], last_point[1]]
            for key, value in self.DummyNodes.items():
                if value == position:
                    randomId = key

            self.DummyNodes[randomId] = position

            sorted_intersection.insert(0, {'id':randomId, 'type': 'node', 'type_1': 'Dummy' ,'distance':0, 'position':position})


        if distance3 > alpha and distance4 > alpha:
            randomId = self.generate_unique_random_number()
            position = [first_point[0], first_point[1]]
            for key, value in self.DummyNodes.items():
                if value == position:
                    randomId = key
            self.DummyNodes[randomId] = position
            sorted_intersection.append({'id':randomId, 'type': 'node','type_1': 'Dummy' , 'distance':distanceMax, 'position':position})
            
        
        return sorted_intersection
    
    def euclidean_distance(self, point1, point2):
        return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)
    
    def addInGraph(self, intersectionFinal, row):
        comments  = False
        FirstNode = True
        for i in range(0,len(intersectionFinal)):
            if comments:
                print("*************************************************")
                print("iteration :",i," ",intersectionFinal[i])
                
            if FirstNode:
                idSource = intersectionFinal[i]["id"]
                typeNode = intersectionFinal[i]["type_1"]
                
                color = self.getColorNode(typeNode)
                self.G.add_node(int(idSource), type="node", position = intersectionFinal[i]['position'], col=color)
                if comments:
                    print("Added node",idSource )
                FirstNode = False
                continue
            else:
                idDist = intersectionFinal[i]["id"]
                color = self.getColorNode(intersectionFinal[i]["type_1"])
                self.G.add_node(int(idDist), type="node", position = intersectionFinal[i]['position'], col=color)

                self.G.add_edge(idSource, idDist, type="pipe",idEdge = row['id'])
                idSource = idDist

        return True
    
    def getColorNode(self, nodeType):
        
        color = 'black'
        if nodeType == "Manholes":
            color = 'red'
        elif nodeType == "Structures":
            color = 'springgreen'
        elif nodeType == "Pumps":
            color = 'yellow'
        elif nodeType == "Fittings":
            color = 'cyan'
        elif nodeType == "TreatmentPlant":
            color = 'black'
        elif nodeType == "Accessories":
            color = 'orange'
        elif nodeType == "Appariel":
            color = "blue"
        elif nodeType == "Deversoir":
            color = "bisque"
        elif nodeType == "PosteRefoulement":
            color = "peru"
        elif nodeType == "Dummy":
            color = 'violet'

        return color
            
    def convert_to_serializable(self, obj):
        if isinstance(obj, np.int64):
            return int(obj)
        return obj

    def save_edges_in_json(self):
        for source, target, attributes in self.G.edges(data=True):
            
            edge_attributes = self.shp.get_pipe_attributes(attributes['idEdge'])
            edge_attributes["properties"]["sourceNode"] = source
            edge_attributes["properties"]["targetNode"] = target
            self.edges_data.append(edge_attributes)
        self.replace_NaN_by_None_edges()
        with open(self.edges_output_file, 'w') as edge_json_file:
            json.dump(self.edges_data, edge_json_file, indent=4)

    def save_nodes_in_json(self):
        self.shp.find_all_components_points_path()
        self.shp.read_all_components_points_paths()
        self.shp.get_attributes_components_points()
        for node, attributes in self.G.nodes(data=True):
            
            source_1= self.mapping_color_components[attributes["col"]]
            if source_1 != "Dummy":
                node_attributes = self.shp.get_node_attributes(source_1, node)
            else:
                node_attributes = self.get_attributes_Dummy(node)

            node_attributes["properties"]["id"] = node
            node_attributes["properties"]["source_1"] = source_1
            self.nodes_data.append(node_attributes)
            
        self.replace_NaN_by_None_nodes()  # Call your method to replace NaN with None if needed
        with open(self.nodes_output_file, 'w') as node_json_file:
            json.dump(self.nodes_data, node_json_file, indent=4)

    def replace_NaN_by_None_nodes(self):
        # Replace NaN values with None before creating the JSON representation
        for feature in self.nodes_data:
            for key, value in feature["properties"].items():
                if isinstance(value, float) and math.isnan(value):
                    feature["properties"][key] = None
        return True
    
    def replace_NaN_by_None_edges(self):
        # Replace NaN values with None before creating the JSON representation
        for feature in self.edges_data:
            for key, value in feature["properties"].items():
                if isinstance(value, float) and math.isnan(value):
                    feature["properties"][key] = None
        return True
    
    def get_attributes_Dummy(self, idNode):
        return {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": self.DummyNodes[idNode]
                },
                "properties": {
                    "id": idNode,
                    "components":"Dummy"
                }
            }